# **Chains**

chains are a fundamental concept that allows you to execute complex tasks in a structured and efficient way.

## **Why Chains?**

Chains are invaluable due to their capacity to effortlessly blend diverse components, shaping a singular and coherent application. Through the creation of chains, multiple elements can seamlessly come together. Imagine this scenario: a chain is crafted to take in user input, polish it using a PromptTemplate, and subsequently pass on this refined response to a large language model (LLM). This streamlined process not only simplifies but also enriches the overall functionality of the system. In essence, chains serve as the linchpin, seamlessly connecting different parts of the application and enhancing its capabilities.


In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade langchain langchain_community langchain_aws

In [ ]:
!pip uninstall -y numpy pandas
!pip install --no-cache-dir numpy==1.26.4 pandas

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.3.0
Uninstalling pandas-2.3.0:
  Successfully uninstalled pandas-2.3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 134.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 226.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 228.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatib

In [1]:
import os
from google.colab import userdata
os.environ["AWS_ACCESS_KEY_ID"] = userdata.get('AWS_ACCESS_KEY_ID')
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get('AWS_SECRET_ACCESS_KEY')
os.environ["AWS_DEFAULT_REGION"] = userdata.get('AWS_DEFAULT_REGION')

In [3]:
import pandas as pd
df = pd.read_csv("/content/chains/employee.csv")

In [4]:
df.head()

,Employee ID,Employee Name,Designation,Tools Used,Date of Birth,Salary,Hire Date,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,1,John Doe,Software Engineer,"Eclipse, Git, JIRA",15-03-1985,"$75,000",20-06-2010,NaN,NaN,NaN,NaN,NaN
1,2,Jane Smith,UI/UX Designer,"Figma, Adobe XD, Sketch",22-08-1990,"$55,000",10-02-2019,NaN,NaN,NaN,NaN,NaN
2,3,Alice Brown,Database Administrator,"MySQL, MongoDB, Oracle",10-11-1988,"$60,000",05-04-2015,NaN,NaN,NaN,NaN,NaN
3,4,Bob White,DevOps Engineer,"Jenkins, Docker, Kubernetes",02-04-1980,"$80,000",15-09-2013,NaN,NaN,NaN,NaN,NaN
4,5,Charlie Black,QA Engineer,"Selenium, JUnit, TestRail",18-07-1992,"$65,000",30-10-2020,NaN,NaN,NaN,NaN,NaN


# **LLM Chain** - **The simplest chain**

The LLMChain is a foundational system that includes a PromptTemplate, an OpenAI model (such as a Large Language Model or a ChatModel), and optionally, an output parser. It operates by transforming input parameters using the PromptTemplate into a coherent prompt, which is then fed into the model. The resulting output is further refined and formatted into a usable form by the OutputParser, if provided. This structured approach ensures effective utilization of language models for various applications, enhancing their functionality and utility.

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [6]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    temperature=0.5
)

In [7]:
prompt = ChatPromptTemplate.from_template(
    "What are the tools that need to be learned to earn the {designation}?,provide me one tool"
)

In [8]:
chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-8-1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [9]:
designation = "Devops Engineer"
chain.run(designation)

/tmp/ipython-input-9-4066474957.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(designation)


' To become a DevOps Engineer, there are several tools and technologies that you should be familiar with, as DevOps is all about using automation tools to improve the development process and infrastructure management. Here are some of the most commonly used tools in the DevOps field:\n\n1. Version Control Systems (VCS): Git is the most popular version control system used in DevOps. It allows for efficient collaboration and management of codebases.\n2. Continuous Integration (CI) and Continuous Delivery (CD) tools: Jenkins, Travis CI, CircleCI, and GitLab CI/CD are popular tools for automating the building, testing, and deployment of code.\n3. Infrastructure as Code (IaC) tools: Terraform, Ansible, and Puppet are popular tools for managing infrastructure using code.\n4. Containerization tools: Docker and Kubernetes are widely used for containerizing applications and managing containerized environments.\n5. Configuration Management tools: Chef, Puppet, and Ansible are popular tools for m

# **SimpleSequentialChain**

Simple Sequential Chains allow for a single input to undergo a series of coherent transformations, resulting in a refined output. This sequential approach ensures systematic and efficient handling of data, making it ideal for scenarios where a linear flow of information processing is essential

In [10]:
# SimpleSequentialChain
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [11]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What are the tools that need to be learned to earn the {designation}?,provide me one tool"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [12]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    tool:{tool_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [13]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [14]:
overall_simple_chain.run(designation)



> Entering new SimpleSequentialChain chain...
 To become a successful DevOps Engineer, you'll need to have a solid foundation in various tools, technologies, and practices that enable continuous integration, continuous delivery, and continuous deployment. Here are some of the most commonly used tools in the DevOps field:

1. Version Control Systems (VCS): Git is the most popular version control system used in DevOps. You'll need to learn how to use Git for source code management, branching, merging, and collaborating with team members.

2. Continuous Integration (CI) and Continuous Delivery (CD) tools: Jenkins, Travis CI, CircleCI, and GitLab CI are popular CI/CD tools. You'll need to learn how to set up pipelines, configure builds, and deploy applications automatically.

3. Containerization: Docker is the most widely used containerization platform. You'll need to learn how to create and manage Docker images, containers, and networks.

4. Infrastructure as Code (IaC) tools: Terraform

' Become a proficient DevOps Engineer by mastering Git for version control, Jenkins/Travis CI/CircleCI/GitLab CI for CI/CD, Docker for containerization, Terraform/Ansible/Puppet for IaC, Ansible/Puppet/Chef for configuration management, and Prometheus/Grafana/ELK Stack for monitoring/logging, as well as Kubernetes for container orchestration. Start with Git for a strong foundation.'

# **SequentialChain**

A sequential chain is a chain that combines various individual chains, where the output of one chain serves as the input for the next in a continuous sequence. It operates by running a series of chains consecutively.

In [15]:
# SequentialChain
from langchain.chains import SequentialChain

In [16]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following designation"
    "\n\n{Designation}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="Designation_description"
                    )


In [17]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following description in 1 sentence:"
    "\n\n{Designation_description}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [18]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What are the promgramming language that need to be learned for this designation in one line:\n\n{Designation}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="programming_language"
                      )

In [19]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a short follow up response to the following "
    "Write a short summary about the programming language:"
    "\n\nSummary: {summary}\n\nLanguage: {programming_language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [20]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Designation"],
    output_variables=["Designation_description","programming_language", "summary","followup_message"],
    verbose=True
)

In [21]:
designation = df.Designation[2]
overall_chain(designation)

/tmp/ipython-input-21-3036767809.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(designation)




> Entering new SequentialChain chain...

> Finished chain.


{'Designation': 'Database Administrator',
 'Designation_description': ' A Database Administrator is a tech professional responsible for designing, installing, and maintaining databases to ensure data security, integrity, and optimal performance. They use SQL and other tools to manage complex data systems and troubleshoot issues.',
 'programming_language': " To excel as a Database Administrator, you should master SQL (Structured Query Language) for database design, creation, and management, along with familiarity in a scripting language like PowerShell or Python for automation tasks, and optional knowledge of a specific DBMS vendor's proprietary language, such as PL/SQL for Oracle or T-SQL for Microsoft SQL Server.",
 'summary': ' A Database Administrator is a tech expert who designs, installs, and maintains databases using SQL and other tools to ensure data security, integrity, and optimal performance.',
 'followup_message': " In addition to the summary, it's important to note that SQL

# **Router Chain**

The Router Chain is used for complicated tasks. If we have multiple subchains, each of which is specialized for a particular type of input, we could have a router chain that decides which subchain to pass the input to.

In [22]:
software_engineer_template = """You are a highly skilled software engineer. \
You excel at addressing queries about programming and software development in a clear
and straightforward manner. \
When faced with a challenge outside your expertise, you candidly \
acknowledge the gap in your knowledge. \

Here is a question:
{input}"""


database_administrator_template = """You are a highly skilled database administrator.\
You excel at answering questions about databases in a clear and precise manner, \
making even complex topics accessible to anyone.\
When faced with a query or issue you're unfamiliar with, \
you candidly admit that you need to look into it further, valuing accuracy over conjecture.\

Here is a question:
{input}"""

data_scientist_template = """You are a highly skilled data scientist.\
You excel at interpreting complex datasets and presenting insights in a \
straightforward and comprehensible manner.\
When confronted with a query beyond your expertise, you acknowledge \
your limitations and advocate for further research or consultation.

Here is a question:
{input}"""


Machine_learning_engineer_template = """ You are a highly skilled Machine Learning Engineer.\
You excel at answering questions about machine learning algorithms and models in a clear and succinct manner.\
When confronted with a query outside your expertise, you candidly admit that you don't have the answer. \

Here is a question:
{input}"""

In [23]:
prompt_infos = [
    {
        "name": "software_engineer",
        "description": "Good for answering questions about software_engineer",
        "prompt_template": software_engineer_template
    },
    {
        "name": "database_administrator",
        "description": "Good for answering database_administrator questions",
        "prompt_template": database_administrator_template
    },
    {
        "name": "data_scientist",
        "description": "Good for answering data_scientist questions",
        "prompt_template": data_scientist_template
    },
    {
        "name": "Machine_learning_engineer",
        "description": "Good for answering Machine_learning_engineer questions",
        "prompt_template": Machine_learning_engineer_template
    }
]

In [24]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [25]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [26]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [27]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [28]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [29]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipython-input-29-2333002322.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [30]:
chain.run("What are the various categories of software?")



> Entering new MultiPromptChain chain...
software_engineer: {'input': 'What are the different types or categories of software?'}
> Finished chain.


" I'm glad you asked about the different types or categories of software! There are several common classifications based on various factors such as functionality, development approach, and intended use. Here are some of the most common types:\n\n1. System Software: System software is the foundation upon which other applications run. It includes operating systems, device drivers, and utility programs. System software manages hardware resources and provides common services for other applications.\n2. Application Software: Application software is designed to perform specific tasks for end-users. Examples include productivity software like Microsoft Office, media players like VLC, and web browsers like Google Chrome. Application software is what users interact with directly to accomplish their goals.\n3. Programming Software: Programming software, also known as development tools or integrated development environments (IDEs), is used to create, test, and debug other software. Examples inclu

In [31]:
chain.run("What are the differences between supervised and unsupervised learning")



> Entering new MultiPromptChain chain...
Machine_learning_engineer: {'input': 'What are the key differences between supervised and unsupervised learning in machine learning?'}
> Finished chain.


" I'd be happy to help answer that question!\n\nSupervised learning and unsupervised learning are two broad categories of machine learning algorithms. The main difference between the two lies in the type of data they use and the goals they aim to achieve.\n\nIn supervised learning, the algorithm is trained on a labeled dataset, which means that the input data comes with corresponding output labels. The algorithm learns to map inputs to outputs by finding patterns in the data. Once trained, the algorithm can then make predictions on new, unseen data based on the patterns it has learned. Examples of supervised learning algorithms include linear regression, logistic regression, support vector machines, and neural networks.\n\nOn the other hand, unsupervised learning algorithms are used when the input data is not labeled. The algorithm identifies patterns and structures in the data on its own, without any explicit guidance. The goal of unsupervised learning is often to discover hidden rela

# **Let's Do an Activity**

## **Objective**

Practice using chains with language models and structured output parsing to handle complex tasks efficiently.

## **Scenario**

You are tasked with building a system to assist users in understanding various technical concepts related to software engineering roles. Your goal is to use different chains to process user queries, transform them using prompt templates, and extract specific information using output parsers.

## **Steps**

* Define a Prompt Template
* Prepare Sample Queries
* Implement LLM Chains
* Output Parsing
* Interactive Chain Execution
* Evaluate